<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: stealthstresser2
#### File originally retrieved from: http://4lz5rmnkd6f63tmm.onion/db/stealthstresser.sql

## * Libraries that I use to analyse data

In [1]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [2]:
dumpfile='stealthstresser.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [3]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

-- phpMiniAdmin dump 1.8.120510
-- Datetime: 2013-09-07 04:43:18
-- Host: Knock8351.db.11414902.hostedresource.com
-- Database: Knock8351

/*!40030 SET NAMES utf8 */;
/*!40030 SET GLOBAL max_allowed_packet=16777216 */;

DROP TABLE IF EXISTS `API`;
CREATE TABLE `API` (
`UDP` text NOT NULL
) ENGINE=MyISAM DEFAULT CHARSET=latin1;

/*!40000 ALTER TABLE `API` DISABLE KEYS */;
INSERT INTO `API` VALUES ('http://212-hosting.net/send.php?key=sad21ad(@*nand&host=[host]&time=[time]&port=[port] http://212-hosting.net/send.php?key=792013&host=[host]&time=[time]&port=[port]');
/*!40000 ALTER TABLE `API` ENABLE KEYS */;

DROP TABLE IF EXISTS `blacklist`;
CREATE TABLE `blacklist` (
`ID` int(11) NOT NULL auto_increment,
`IP` varchar(15) NOT NULL,
`note` text NOT NULL,
PRIMARY KEY  (`ID`)
) ENGINE=MyISAM AUTO_INCREMENT=16 DEFAULT CHARSET=latin1;

/*!40000 ALTER TABLE `blacklist` DISABLE KEYS */;
INSERT INTO `blacklist` VALUES ('10','96.61.18.81','magic_marathon'),('11','69.131.10.247','bitch'),('12','69

## 2. Listing tables that have content inserted into the dump file

In [4]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [5]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 API
2 blacklist
3 fe
4 gateway
5 logs
6 news
7 payments
8 plans
9 skype_api
10 users


## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [6]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

In [7]:
# For well formed SQL database dumps!
def read_inserted_table_entirein1line(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert operation)
            
            for record in data:
                try:
                    newline = record.strip('()') #Step2:remove parenthesis
                    newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                    newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                    newline=newline.replace('\'','') #Step5: remove single quotes
                    newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                    sio.write(newline)
                    sio.write("\n")
                except IndexError:
                    pass
                
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

## * Converter functions for formatting content of columns
Attention, it is better to perform the conversion based on a series than a string (http://stackoverflow.com/questions/42462906/pandas-read-csv-converters-performance-issue)

In [8]:
def tolowercase(series):
    return series.str.lower()

In [9]:
def strip_singlequote(series):
    return series.str.strip('\'')

In [10]:
def timestamp2datetime(series):
    return  pd.to_datetime(series,unit='s',errors='coerce')

In [11]:
def datetimestring2datetime(series):
    return  pd.to_datetime(series, format='%d-%m-%Y %H:%M',errors='coerce')  

In [12]:
def datetimeinvertedstring2datetime(series):
    return pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [13]:
def datetimeampm2datetime(series):
    return  pd.to_datetime(series,format='%d-%m-%Y %I:%M:%S %p',errors='coerce')

In [14]:
def datetimemdyampm2datetime(series):
    return  pd.to_datetime(series, format='%m-%d-%Y %I:%M:%S %p',errors='coerce') 

In [15]:
def strip_parenthesisandsemicolon(series):
    return series.str.strip(');')

In [16]:
def strip_insertintologs(series):
    return int(series.str.strip('INSERT INTO `logs` VALUES ('))

In [17]:
def strip_tab(series):
    return series.str.strip('\t')

In [18]:
def friendsenemies_type(series):
    return series.str.replace('f','friend').replace('e','enemy')
    

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'API'

### How this table looks like without modification?

In [19]:
tablename='API'

pd.read_csv(read_inserted_table_entirein1line(dumpfile,tablename),delimiter=",",error_bad_lines=False)

,http://212-hosting.net/send.php?key=sad21ad(@*nand&host=[host]&time=[time]&port=[port] http://212-hosting.net/send.php?key=792013&host=[host]&time=[time]&port=[port]


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type/converter: 
        o 
    - On the column name:
        o 
    - Split columns:
        o 
    - Add required columns:
        o 

## 6.2. Table: 'blacklist'

### How this table looks like without modification?

In [20]:
tablename='blacklist'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,10,96.61.18.81,magic_marathon
0,11,69.131.10.247,bitch
1,12,69.131.16.144,bitch
2,13,69.131.23.212,bitch
3,14,69.131.5.130,you suck dick


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: blacklist
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o id
        o ip
        o note
    - Split columns:
        o 

### Modifying table: 'blacklist'

In [21]:
# Loading the correct table name
columns = ['id','ip','note']
df_blacklist = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False, names=columns)

# Converting columns

# Changing column names

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_blacklist.head(10)

,id,ip,note
0,10,96.61.18.81,magic_marathon
1,11,69.131.10.247,bitch
2,12,69.131.16.144,bitch
3,13,69.131.23.212,bitch
4,14,69.131.5.130,you suck dick


## 6.3. Table: 'fe'

### How this table looks like without modification?

In [22]:
tablename='fe'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,1,12,e,110.175.100.136,Auspride018
0,2,13,e,94.197.11.24,NaN
1,3,8,f,50.138.30.101,Nely
2,4,8,f,50.55.247.24,Brittany
3,5,8,f,98.94.78.64,channa
4,6,8,f,108.210.196.40,Hector
5,7,8,f,70.100.180.240,Kimmie
6,8,8,f,50.33.227.18,Karissa
7,9,8,f,108.214.117.105,Lilla
8,10,26,e,85.229.8.249,Jimman
9,11,26,e,213.114.138.6,dillen


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: friendsenemies
#### Q3: Are there modifications required? 
    - On the table name: 
        o fe -> friendsenemies
    - On the column type: 
        o type -> friendsenemies_type() 
    - On the column name:
        o 
    - Add required columns:
        o id
        o ip
        o note
        o userid 
        o type
    - Split columns:
        o 

### Modifying table: 'fe'

In [23]:
# Loading the correct table name
columns=['id','userid','type','ip','note']
df_friendenemies = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False,names=columns)

# Converting columns
df_friendenemies['type']=friendsenemies_type(df_friendenemies['type'])

# Changing column names

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_friendenemies.head(10)

,id,userid,type,ip,note
0,1,12,enemy,110.175.100.136,Auspride018
1,2,13,enemy,94.197.11.24,NaN
2,3,8,friend,50.138.30.101,Nely
3,4,8,friend,50.55.247.24,Brittany
4,5,8,friend,98.94.78.64,channa
5,6,8,friend,108.210.196.40,Hector
6,7,8,friend,70.100.180.240,Kimmie
7,8,8,friend,50.33.227.18,Karissa
8,9,8,friend,108.214.117.105,Lilla
9,10,26,enemy,85.229.8.249,Jimman


## 6.4. Table: 'gateway'

### How this table looks like without modification?

In [24]:
tablename='gateway'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,separk6112@inha.ac.kr


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: gateways
#### Q3: Are there modifications required? 
    - On the table name: 
        o gateway -> gateways
    - On the column type: 
        o  
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'gateway'

In [25]:
# Loading the correct table name
columns=['email']
df_gateways = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False,names=columns)

# Converting columns

# Changing column names

# Showing some lines after adapt the table
df_gateways.head(10)

,email
0,separk6112@inha.ac.kr


## 6.5. Table: 'logs'

### How this table looks like without modification?

In [26]:
tablename='logs'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False)

,dominerd,99.109.176.129,8080,70,CHARGEN,1378262797
0,dominerd,107.2.198.58,8080,400,DRDOS,1378263237
1,Rigour,123.211.154.254,80,0,DRDOS,1378307552
2,Rigour,198.27.75.55,25565,0,DRDOS,1378326488
3,Rigour,82.45.73.76,53,0,DRDOS,1378334408
4,Rigour,178.33.143.97,80,0,DRDOS,1378343669
5,Rigour,198.20.101.120,80,0,DRDOS,1378404967
6,Rigour,213.107.159.172,53,0,DRDOS,1378413383
7,Rigour,198.27.75.55,25565,0,DRDOS,1378416760
8,Rigour,213.107.159.172,53,0,DRDOS,1378419772
9,Rigour,199.83.50.45,25565,3600,DRDOS,1378422239


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: attacks
#### Q3: Are there modifications required? 
    - On the table name: 
        o logs -> attacks
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o 
    - Add required columns:
        o username
        o targetip
        o port
        o duration
        o type
        o date

        o id
        o userid
        o targeturl
        

    - Split columns:
        o 

### Modifying table: 'logs'

In [27]:
# Loading the correct table name
columns=['username','targetip','port','duration', 'type', 'date']
df_attacks = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False,names=columns)

# Converting columns
df_attacks['date']=timestamp2datetime(df_attacks['date'])

# Changing column names

# Adding missing columns
df_attacks['id']=np.nan
df_attacks['userid']=np.nan
df_attacks['targeturl']=''

# Showing some lines after adapt the table
df_attacks.head(10)

,username,targetip,port,duration,type,date,id,userid,targeturl
0,dominerd,99.109.176.129,8080,70,CHARGEN,2013-09-04 02:46:37,NaN,NaN,
1,dominerd,107.2.198.58,8080,400,DRDOS,2013-09-04 02:53:57,NaN,NaN,
2,Rigour,123.211.154.254,80,0,DRDOS,2013-09-04 15:12:32,NaN,NaN,
3,Rigour,198.27.75.55,25565,0,DRDOS,2013-09-04 20:28:08,NaN,NaN,
4,Rigour,82.45.73.76,53,0,DRDOS,2013-09-04 22:40:08,NaN,NaN,
5,Rigour,178.33.143.97,80,0,DRDOS,2013-09-05 01:14:29,NaN,NaN,
6,Rigour,198.20.101.120,80,0,DRDOS,2013-09-05 18:16:07,NaN,NaN,
7,Rigour,213.107.159.172,53,0,DRDOS,2013-09-05 20:36:23,NaN,NaN,
8,Rigour,198.27.75.55,25565,0,DRDOS,2013-09-05 21:32:40,NaN,NaN,
9,Rigour,213.107.159.172,53,0,DRDOS,2013-09-05 22:22:52,NaN,NaN,


## 6.6. Table: 'news'

### How this table looks like without modification?

In [28]:
tablename='news'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,29,ToS ( Terms of Service,1 By purchasing Stealth Stresser you agree to our ToS.<br> 2 We are not responsible for how ever you use this stresser<br> 3 You may not share your account if you will your account will be closed!<br> 4 We have the right to close any account we want or shut down the project at any time.<br> 5No Refunds or chargebacks All sales are final.<br> 6We have the right to change the ToS anytime we want.<br>Disrespect will result in no more support & no more updates. You\ve been warned. <br> <br> <br> <br> <br> <br> <br>1353597966(36Later on the stresser will hit harderWe are going to add a dedicated server soon so if its hitting it will hit harder when we add that server.1373527742(39It should hit alot hard nowWe have added a 1GBPS dedicated server so it should hit harder.1373910351(46READHi everybody my api that I got off my friend doesnt even work now so I need to wait till he replys so I can resolve this or something. So in result it wont hit till I get it fixed.1375985926(47READApi is working now the power is back up and running.1375988481(49ToS ( Terms of Service


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.7. Table: 'payments'

### How this table looks like without modification?

In [30]:
tablename='payments'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(30)

,1,18,49,824,dowley.Trevor@Yahoo.com,2SW7239274641070A,1374076531
0,2,2,38,832,lccentore@gmail.com,2PG62971JF453412L,1374124380
1,3,2,38,842,mw3clan27@gmail.com,50585789JY281032A,1374436073
2,4,7,41,865,aliciastrick01@hotmail.com,7KE93157UM496071B,1374689128
3,5,45,52,1473,tagsalestorect@gmail.com,4TY36479KC5592700,1378146737


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: payments
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o 
    - Add required columns:
        o id
        o amountpaid
        o planid
        o userid
        o paymentemail
        o tid
        -
        o username
    - Split columns:
        o 

### Modifying table: 'payments'

In [33]:
# Loading the correct table name
columns=['id','amountpaid','planid','userid','paymentemail','tid','date','username']
df_payments = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False,names=columns)

# Converting columns
df_payments['date'] = timestamp2datetime(df_payments['date']) 

# Changing names of columns

# Creating empty columns (with "" [for future string] or np.nan [for future float])

# Showing some lines after adapt the table
df_payments.head(10)

,id,amountpaid,planid,userid,paymentemail,tid,date,username
0,1,18,49,824,dowley.Trevor@Yahoo.com,2SW7239274641070A,2013-07-17 15:55:31,NaN
1,2,2,38,832,lccentore@gmail.com,2PG62971JF453412L,2013-07-18 05:13:00,NaN
2,3,2,38,842,mw3clan27@gmail.com,50585789JY281032A,2013-07-21 19:47:53,NaN
3,4,7,41,865,aliciastrick01@hotmail.com,7KE93157UM496071B,2013-07-24 18:05:28,NaN
4,5,45,52,1473,tagsalestorect@gmail.com,4TY36479KC5592700,2013-09-02 18:32:17,NaN


## 6.8. Table: 'plans'

### How this table looks like without modification?

In [34]:
tablename='plans'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,42,Bronze 3 Months,3GBPS Bronze 3 Months,1200,Months,3,25
0,38,1 day trial,3GBPS 1 day trail,300,Days,1,2
1,41,Bronze Monthly,6GBPS Bronze Monthly,1200,Days,30,10
2,44,Bronze Lifetime,6GBPS Bronze Lifetime,1200,Years,20,35
3,45,Silver Monthy,6GBPS Silver Monthly,2400,Days,30,15
4,46,Silver 3 Months,6GBPS Silver 3 Months,2400,Months,3,30
5,62,Platinum Lifetime,6GBPS Platinum Lifetime,7200,Years,20,75
6,48,Silver Lifetime,6GBPS Silver Lifetime,2400,Years,20,45
7,49,Gold Monthly,6GBPS Gold Monthly,3600,Days,30,20
8,50,Gold 3 Months,6GBPS Gold 3 Months,3600,Months,3,35
9,61,Special Lifetime,6GBPS Special Lifetime,1000,Years,20,30


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o planid
        o planname 
        o plandescr
        o maxboottime
        o expiration1
        o expiration2
        o price
        -
        o concurrecy
    - Split columns:
        o 

### Modifying table: 'plans'

In [37]:
# Loading the correct table name
columns=['planid','planname', 'plandescr', 'maxboottime','expiration1', 'expiration2','price','concurrency']
df_plans = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False, names=columns)

# Converting columns

# Changing names of columns

# Creating empty columns (with "" [for future string] or np.nan [for future float])

# Showing some lines after adapt the table
df_plans.head(10)

,planid,planname,plandescr,maxboottime,expiration1,expiration2,price,concurrency
0,42,Bronze 3 Months,3GBPS Bronze 3 Months,1200,Months,3,25,NaN
1,38,1 day trial,3GBPS 1 day trail,300,Days,1,2,NaN
2,41,Bronze Monthly,6GBPS Bronze Monthly,1200,Days,30,10,NaN
3,44,Bronze Lifetime,6GBPS Bronze Lifetime,1200,Years,20,35,NaN
4,45,Silver Monthy,6GBPS Silver Monthly,2400,Days,30,15,NaN
5,46,Silver 3 Months,6GBPS Silver 3 Months,2400,Months,3,30,NaN
6,62,Platinum Lifetime,6GBPS Platinum Lifetime,7200,Years,20,75,NaN
7,48,Silver Lifetime,6GBPS Silver Lifetime,2400,Years,20,45,NaN
8,49,Gold Monthly,6GBPS Gold Monthly,3600,Days,30,20,NaN
9,50,Gold 3 Months,6GBPS Gold 3 Months,3600,Months,3,35,NaN


## 6.9. Table: 'skype_api'

### How this table looks like without modification?

In [38]:
tablename='skype_api'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,2,93.114.45.84


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.10. Table: 'users'

### How this table looks like without modification?

In [40]:
tablename='users'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(30)

,871,swatted,dd50e050d7759c30f82725a4eae46844701b3c91,jonathanfoster20@gmail.com,0,0.1,0.2,0.3,Teeqo
0,796,KnockSteady,a9a61999d7570ae1ae5107c7eee23692823c64b2,soomindpark@me.com,1,52,2008977261,0,NaN
1,1476,Cred,834593265b5678ff6848dfd0d9cb401df71fee96,Credasityy@gmail.com,0,0,0,0,Error
2,805,yaseen46,04d354679cb9d2c822805bf68eb4d1f08e96a5b4,yourmom@youhome.com,0,0,0,0,NaN
3,842,patrick3233,4c9693c7531ce0a6a0dba58c87418747e60a75ef,dragon12@live.dk,0,0,1374522473,0,Codex_Legends
4,812,bigdaddy69,94ad646af91dd2afd3754776cf31a993955a6b5c,brettmabry1@gmail.com,0,52,2004832145,0,NaN
5,828,Degree,8dd197c39890842518bef914380e59135c0ee592,vitalxeffectz@gmail.com,0,0,0,0,Crook
6,819,Storytellin,6402fa06a9740c51b3a946085d60059819e4a312,Storytellin@story.com,0,0,0,0,NaN
7,872,KingRecon,985e8f25c0a698926bee3364782af93c0cd5872f,SuPraSnip@gmail.com,0,0,0,0,NaN
8,835,Rigour,15702b4fa7ef2ed38e2b4f6aad7dee6824441f8b,anonymousrigour@gmail.com,1,52,2006192884,0,NaN
9,832,TakeoverXbox,84d24425d9fa012817c77da188f95d1beb3d6434,lccentore@gmail.com,0,0,0,0,Teeqo


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: users
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o expire -> timestamp2datetime()
    - On the column name:
        o userid
        o username
        o password
        o useremail 
        o unknown1
        o unknown2
        o expire
        o unknown3
        o unknown4
        o planid
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'users'

In [41]:
# Loading the correct table name
columns = ['userid','username','password','useremail','unknown1','unknown2','expire','unknown3','unknown4','planid']
df_users = pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False,names=columns)

# Converting columns
df_users['expire'] = timestamp2datetime(df_users['expire']) 

# Changing names of columns

# Creating empty columns (with "" [for future string] or np.nan [for future float])

# Showing some lines after adapt the table
df_users.head(10)

,userid,username,password,useremail,unknown1,unknown2,expire,unknown3,unknown4,planid
0,871,swatted,dd50e050d7759c30f82725a4eae46844701b3c91,jonathanfoster20@gmail.com,0,0,1970-01-01 00:00:00,0,Teeqo,NaN
1,796,KnockSteady,a9a61999d7570ae1ae5107c7eee23692823c64b2,soomindpark@me.com,1,52,2033-08-30 01:14:21,0,NaN,NaN
2,1476,Cred,834593265b5678ff6848dfd0d9cb401df71fee96,Credasityy@gmail.com,0,0,1970-01-01 00:00:00,0,Error,NaN
3,805,yaseen46,04d354679cb9d2c822805bf68eb4d1f08e96a5b4,yourmom@youhome.com,0,0,1970-01-01 00:00:00,0,NaN,NaN
4,842,patrick3233,4c9693c7531ce0a6a0dba58c87418747e60a75ef,dragon12@live.dk,0,0,2013-07-22 19:47:53,0,Codex_Legends,NaN
5,812,bigdaddy69,94ad646af91dd2afd3754776cf31a993955a6b5c,brettmabry1@gmail.com,0,52,2033-07-13 01:49:05,0,NaN,NaN
6,828,Degree,8dd197c39890842518bef914380e59135c0ee592,vitalxeffectz@gmail.com,0,0,1970-01-01 00:00:00,0,Crook,NaN
7,819,Storytellin,6402fa06a9740c51b3a946085d60059819e4a312,Storytellin@story.com,0,0,1970-01-01 00:00:00,0,NaN,NaN
8,872,KingRecon,985e8f25c0a698926bee3364782af93c0cd5872f,SuPraSnip@gmail.com,0,0,1970-01-01 00:00:00,0,NaN,NaN
9,835,Rigour,15702b4fa7ef2ed38e2b4f6aad7dee6824441f8b,anonymousrigour@gmail.com,1,52,2033-07-28 19:48:04,0,NaN,NaN


<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [ ]:
df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
#df_blacklist=pd.DataFrame(columns=['id','ip','note'])
#df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
#df_gateways=pd.DataFrame(columns=['email'])
#df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
#df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
#df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)